Run pga for a fixed channel matrix and compare the result with waterfilling algorithm output

In [587]:
import numpy as np
import torch
import os
import sys
import importlib
torch.set_default_dtype(torch.float64)
import matplotlib.pyplot as plt

# Get the current working directory
scripts_dir = os.getcwd()
# Go up two levels
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

import src.beam_network_ch_model
importlib.reload(src.beam_network_ch_model)
from src.beam_network_ch_model import Network, pair

import src.utils
importlib.reload(src.utils)
from src.utils import calculate_rate

import src.beamforming
importlib.reload(src.beamforming)
from src.beamforming import wf_algorithm

import src.pga
importlib.reload(src.pga)
from src.pga import fixed_channel_pga, Bnetwork_channel_pga

In [583]:
# For fixed channel
#Setting Network Parameters
num_RX_ant = 2
num_TX_ant = 4
num_scatterers = torch.tensor([[1, 1], [1, 1]])
lam = 0.01
Ant_dist = 0.005
#Setting Locations
TX_locs = [torch.tensor([-10.0, 0.0], dtype=torch.float64), torch.tensor([0.0, 0.0], dtype=torch.float64)]
RX_locs = [torch.tensor([-10.0, -10.0], dtype=torch.float64), torch.tensor([0, -10.0], dtype=torch.float64)]
SC_locs = np.array([[torch.tensor([[-20.0, -5.0]], dtype=torch.float64), torch.tensor([[10.0, -5.0]], dtype=torch.float64)],
                     [torch.tensor([[-20.0, -5.0]], dtype=torch.float64), torch.tensor([[10.0, -5.0]], dtype=torch.float64)]])

N = Network()
N.num_RX_ant = num_RX_ant
N.num_TX_ant = num_TX_ant
N.num_scatterers = num_scatterers
N.lam = lam
N.Ant_dist = Ant_dist
N.TX_locs = TX_locs
N.RX_locs = RX_locs
N.SC_locs = SC_locs
N.rand_ph = 1

p = pair(N, 0, 0)

Br = p.calculate_Br()
Bt = p.calculate_Bt()
A = p.calculate_A()
H = Br @ A @ Bt.conj().T

In [584]:
wf = wf_algorithm(H=H, Pt=5)
p_aloc = wf.p_allocation()
bf = wf.bf_matrix()
sig1 = bf @ p_aloc @ bf.conj().T
r1 = calculate_rate(H, Sigma)
r1

tensor(0.4282-1.4328e-09j, dtype=torch.complex64, grad_fn=<Log2Backward0>)

In [585]:
f = fixed_channel_pga(H, 5)
Sigma = f.solve()
r2 = calculate_rate(H, Sigma)
r2

tensor(0.4323-7.8656e-09j, dtype=torch.complex64)

Run pga for beam network channel model

In [ ]:
p = pair(N, 0, 0)
bnet = Bnetwork_channel_pga(P=p, PT=5)
bnet.solve()